In [3]:
import pandas
import numpy as np
import sklearn as sk

In [5]:
data = pandas.read_csv('/Users/justinpassek/Downloads/oculi_feature_labels.csv')

In [6]:
perf = pandas.read_csv('/Users/justinpassek/Downloads/creative_performance.csv')
perf['CTR'] = perf['clicks'] / perf['imps']
quantile = perf['CTR'].quantile(.5)
perf['label'] = perf['CTR'].apply(lambda x: 1 if x > quantile else 0)

In [7]:
features = data['description'].str.get_dummies()

In [8]:
agg = features.aggregate('sum',axis=0)

In [9]:
features_to_drop = list(agg[agg < 10].index)

In [10]:
filtered_features = pandas.DataFrame({'encoded':features.drop(columns=features_to_drop).values.tolist()})

In [11]:
encoded_data = data.merge(filtered_features,how='outer',left_index=True,right_index=True)

encoded_data['encoded'] = encoded_data['encoded'].apply(np.array)

In [12]:
data_collapsed = encoded_data[['Creative_ID','encoded']].groupby(['Creative_ID'],as_index=False).sum()

In [13]:
data_labeled = data_collapsed.merge(perf[['Creative_ID','label']],on=['Creative_ID'],how='left')
data_labeled = data_labeled.fillna(0)

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
LR = LogisticRegression(penalty='l1',max_iter=100)

In [16]:
model = LR.fit(
    pandas.DataFrame(data_labeled['encoded'].tolist()).loc[:],
    pandas.DataFrame(data_labeled['label'].tolist()).loc[:].values.ravel()
)

In [17]:
feature_coef = pandas.concat(
    [pandas.DataFrame(features.drop(columns=features_to_drop).columns,columns=['features']),
     pandas.DataFrame(np.transpose(LR.coef_),columns=['coefficients'])],
    axis=1
)

In [353]:
feature_coef[feature_coef['coefficients'] != 0]

,features,coefficients
2,Banner,-0.718113
3,Bed,0.049305
4,Bed frame,0.024015
8,Beige,-0.743051
10,Blue,-0.037754
12,Brand,0.833986
13,Brown,1.607657
15,Circle,-1.346199
16,Comfort,0.431141
17,Company,0.660241
